#### Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Originally written in Mathematica by J. Christopher Edgar. Conversion to Jupyter Notebook by Song Liu.

The authors of this courseware are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.06 The Fourier Transform</font>

# <font color=red>BASICS</font>

### Setup

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special
import matplotlib.patches as patches
import math
import cmath
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b"): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))

unknownfreq1 = 6
unknownmag1 = 3
unknownphase1 =np.pi/2
unknownfreq2 = 4
unknownmag2 = 24
unknownphase2 =np.pi/4
unknownfreq3 = 5
unknownmag3 = 31
unknownphase3 =0


In Lesson 5, we learned how to determine the frequencies (and their associated magnitudes and
phases) composing a timeseries or a spatial image. In this Lesson, we will examine the procedure
most commonly used to transfer data from the time or spatial domain to the frequency domain. This
procedure, called the Fourier Transform, is one of the most important mathematical discoveries. We
will examine how the Fourier Transform works. By the end of this lesson, you will be a sophisticated
Fourier Transform user. You will probably find this lesson easier than the previous lesson, because here
we only need to slightly extend what you already know. But hey, is this extension useful!

## <font color=red>DSP.06.B1) Examining the Frequencies Composing a Timeseries</font>

### <font color=red>DSP.06.B1.a) Using complex exponentials to determine magnitude and phase - doing it the long way</font>

Recall the Euler Identities:

 $e^{i x}$ = cos[x] + $i$sin[x]

and

 $e^{-i x}$ = cos[x] - $i$sin[x]
 
 where $i$ = $\sqrt{-1}$ 

Use complex exponentials to determine the magnitude and phase of this timeseries.

In [ ]:
time = np.arange(0,1,0.001)
timeseries = unknownmag1 * np.sin(2*np.pi * unknownfreq1 * time +unknownphase1) + \
            unknownmag2 * np.sin(2*np.pi * unknownfreq2 * time +unknownphase2) + \
            unknownmag3 * np.sin(2*np.pi * unknownfreq3 * time +unknownphase3)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

In [ ]:
amp_complex = np.exp(2*np.pi * 1j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
round(abs(spectrum),5)

No 1 Hz activity.

Set the frequency $w$ =2 to look for 2 Hz activity.

In [ ]:
amp_complex = np.exp(2*np.pi * 2j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
round(abs(spectrum),5)

No 2 Hz activity.

Set $w$ = 3.

In [ ]:
amp_complex = np.exp(2*np.pi * 3j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
round(abs(spectrum),5)

No 3 Hz activity.

Set $w$ = 4.

In [ ]:
amp_complex = np.exp(2*np.pi * 4j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
spectrum

There is activity at 4 Hz.

Compute the magnitude and phase of 4 Hz activity.

In [ ]:
round(abs(spectrum),5)

In [ ]:
point_x = np.real(spectrum)
point_y = np.imag(spectrum)
phaseradians = round(np.arctan(point_x/point_y),5)
phasedegrees = round(phaseradians * 180 / np.pi,2)
phasedegrees

The output of the 'abs' function indicates the magnitude of 4 Hz activity is 24.
The 'arctan' function indicates that the phase of 4 Hz activity is 45°.
Keep going.

Set $w$ = 5.

In [ ]:
amp_complex = np.exp(2*np.pi * 5j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
spectrum

There is activity at 5 Hz.

Compute the magnitude and phase of the 5 Hz activity.

In [ ]:
round(abs(spectrum),5)

In [ ]:
point_x = np.real(spectrum)
point_y = np.imag(spectrum)
phaseradians = round(np.arctan(point_x/point_y),5)
phasedegrees = round(phaseradians * 180 / np.pi,2)
phasedegrees

The output of the 'abs' function indicates the magnitude of 5 Hz activity is 31.
The 'arctan' function indicates that the phase of 5 Hz activity is 0°.

Set $w$ = 6.

In [ ]:
amp_complex = np.exp(2*np.pi * 6j * time) 
amplitude_complex = timeseries * amp_complex
spectrum = sum(amplitude_complex)*0.001/ 0.5
round(spectrum)

There is 6 Hz activity.

Compute the magnitude and phase of the 6 Hz activity.

In [ ]:
round(abs(spectrum),5)

In [ ]:
point_x = np.real(spectrum)
point_y = np.imag(spectrum)
phaseradians = round(np.arctan(point_x/point_y),5)
phasedegrees = round(phaseradians * 180 / np.pi,2)
phasedegrees

The output of the 'abs' command indicates the magnitude of 6 Hz activity is 3.
The 'arctan' command indicates that the phase of 6 Hz activity is 90°.

We can stop at 6 Hz because we know from the code that this timeseries is composed of just three
frequencies.

### <font color=red>DSP.06.B1.b) Using complex exponentials to determine magnitude and phase - doing it the long way</font>

Here is a timeseries dataset (these data were digitized at 400 Hz, and the length of the epoch is 1
second).

In [ ]:
data = np.array([48.991378028648455, 48.390563096386465, 48.103408911379105,
33.76228709107139, -0.9801374162297645, -22.457288912524753, -4.995630225091868,
25.768288661355413, 36.20355681602837, 32.69058361593039, 29.6263198466567,
12.738622534432494, -24.232999844663066, -47.54729610867588, -31.471245039598905,
-1.5994993303738774, 8.45967261566186, 5.089196245934524, 2.6683725222686094,
-13.111963060087422, -48.56710070811715, -70.02603664946089, -51.838480481405284,
-19.6908203237655, -7.288217090222255, -8.345738706382626, -8.579082224972462,
-22.385448918009907, -56.1575338507298, -76.28788336948384, -57.17326142331313,
-24.528665264847156, -12.068242856194441, -13.496071691571695,
-14.497361711662593, -29.420627343519158, -64.5937361873551, -86.33100210522346,
-68.94084982090804, -38.04355436299916, -27.25615276223265, -30.187321537395036,
-32.432612110631936, -48.260421382370936, -83.93144688803098, -105.70865246743617,
-87.86715190173732, -56.01342133809717, -43.7708763897362, -44.774579468168824,
-44.66547622084393, -57.77457946816883, -90.43992394804833, -109.01342133809717,
-87.86715190173732, -52.70865246743617, -37.26239932971883, -35.26042138237096,
-32.43261211063197, -43.18732153739505, -73.92520032054477, -91.04355436299917,
-68.94084982090803, -33.33100210522348, -17.924688629042958, -16.420627343519158,
-14.497361711662574, -26.496071691571693, -58.73729041450656, -77.52866526484718,
-57.17326142331313, -23.287883369483865, -9.488486292417644, -9.385448918009917,
-8.579082224972439, -21.345738706382605, -53.95726464853442, -72.69082032376548,
-51.8384804814053, -17.026036649460885, -1.898053149805012, -0.11196306008741086,
2.6683725222685757, -7.9108037540654825, -38.2093749426503, -54.59949933037389,
-31.47124503959893, 5.452703891324104, 22.43604771364904, 25.738622534432512,
29.6263198466567, 19.690583615930407, -10.46549074228377, -27.231711338644573,
-4.995630225091891, 30.54271108747524, 45.68891014208236, 46.76228709107139,
48.10340891137909, 35.39056309638647, 2.322330470336312, -17.390852524324405,
1.9643874497834641, 34.785510349549604, 47.46823027167586, 46.41094601666506,
46.01907174782953, 32.01890890257502, -1.8616832487282569, -21.902849363896053,
-2.4019562470204496, 31.008094021686006, 44.67438271814678, 44.93051362557835,
46.10372179638185, 33.83207731063105, 1.7489026552686502, -16.52278324679816,
4.624614753986416, 39.46897456210164, 54.27774947867579, 55.31787863965021,
56.86572402065012, 44.526441656970654, 11.919815442774237, -7.3246924924478085,
12.426708932943637, 45.49679117378558, 58.21476129398312, 56.923628964814085,
55.9869996216546, 41.1048320253671, 5.995338735626045, -15.614053056291343,
2.003873391532952, 33.256883155862496, 44.54719723450923, 42.27580439470092,
40.84681874826837, 25.981522643946608, -8.600655208355814, -29.191270430624954,
-10.10154476231427, 23.019940166497047, 36.503437938904774, 36.665281983069704,
37.815699589791265, 25.576691553203204, -6.432931793393035, -24.602570516788276,
-3.3345237791560685, 31.642026208915922, 46.59048201507564, 47.77677558589566,
49.479197628533846, 37.30752461273516, 4.8884826838602375, -14.139681310038036,
5.866882892538079, 39.241648264699805, 52.32449673697019, 51.46840916135989,
51.04560258424744, 36.76268142287006, 2.342007372824722, -18.48777131249198,
-0.001458345495846336, 32.20326825253826, 44.51972364193224, 43.33663180921074,
43.042727711655445, 29.341176538646966, -4.068954455348979, -23.501234945920984,
-3.2898870787153847, 30.893234987371592, 45.35557381269338, 46.39183691149365,
48.29249530098617, 36.66228113918205, 5.1057504797199424, -12.777322226544069,
8.603652948624067, 43.516243470896484, 58.22446307202845, 58.99857276023159,
60.124628677083656, 47.22320186235163, 13.93432659348393, -6.088530691031108,
12.814948013702008, 44.995254867115015, 56.80914142367121, 54.6256325077892,
52.8315404128707, 37.146071838359525, 1.3017548586137593, -20.965355254920883,
-3.9256053931713497, 26.82478238770549, 37.67766952966369, 35.01764531449422,
33.226985137167105, 18.000537872595043, -16.97224238208146, -38.015207638887546,
-19.472863064548168, 12.973334125562003, 25.62256697084991, 24.763244921011925,
24.68204793665754, 10.982873556975797, -22.727266831566894, -42.84265090495897,
-23.763186253971483, 8.791669442909061, 21.102152813672852, 19.457637458104415,
18.165703100326706, 2.870274939304105, -32.76408725859495, -55.05893215618299,
-38.32923367337307, -8.200799559554465, 1.705116955411782, -2.222803788541154,
-5.580981191600762, -22.636765994999013, -59.64782571307441, -82.87211310716553,
-66.57782545999271, -36.36276278450489, -25.84011528825749, -28.633437432812404,
-30.371196932694183, -45.37130468324919, -79.95843515818618, -100.4712521866426,
-81.26906170532061, -48.04743686437044, -34.52000075806543, -34.40929033684103,
-33.43726042310092, -46.00600167136959, -78.51164016410635, -97.34867543198101,
-76.91439820687832, -42.92302585929681, -29.0868023787925, -29.10701449822058,
-28.66547622084393, -42.10701449822058, -75.7558499371046, -95.92302585929677,
-76.91439820687833, -44.348675431980965, -31.842592605794152,
-33.00600167136962, -33.43726042310093, -47.40929033684101, -81.18904831637745,
-101.04743686437048, -81.26906170532072, -47.47125218664258, -33.28938759987401,
-32.37130468324922, -30.37119693269421, -41.63343743281241, -72.50916284656967,
-89.36276278450494, -66.57782545999271, -29.87211310716568, -12.978778154762315,
-9.636765994999058, -5.5809811916008645, -15.22280378854121, -44.9639306029004,
-61.20079955955448, -38.329233673373174, -2.0589321561830296, 13.90496029971719,
15.870274939304055, 18.16570310032667, 6.457637458104333, -25.566894744639285,
-44.20833055709088, -23.763186253971533, 10.157349095040992, 23.941780726745318,
23.982873556975754, 24.68204793665754, 11.763244921011895, -21.04648058746226,
-40.02666587443802, -19.472863064548225, 14.984792361112445, 29.696805176230686,
31.000537872595014, 33.22698513716708, 22.017645314494118, -8.991378028648448,
-26.17521761229449, -3.9256053931714834, 32.03464474507901, 47.97080241692585,
50.14607183835946, 52.83154041287066, 41.62563250778909, 10.140093865359061,
-8.004745132884999, 12.814948013702008, 46.91146930896887, 60.603374151796054,
60.22320186235163, 60.12462867708366, 45.998572760231504, 11.555415513716316,
-9.483756529103516, 8.603652948624067, 40.22267777345599, 51.77479803803213,
49.66228113918212, 48.29249530098616, 33.39183691149363, -1.3134737456187269,
-22.106765012628383, -3.289887078715366, 29.498765054079005, 42.60009310296318,
42.34117653864695, 43.042727711655445, 30.33663180921075, -2.14932391637994,
-20.79673174746178, -0.0014583454958994047, 34.51222868750792, 49.01105493113683,
49.762681422870116, 51.04560258424739, 38.46840916135988, 5.655449178658077,
-13.758351735300199, 5.866882892538083, 38.860318689961964, 51.55753024217239,
50.30752461273516, 49.47919762853388, 34.77677558589567, -0.0785655432365231,
-21.357973791083996, -3.3345237791560685, 28.39742948321176, 40.236115764919134,
38.576691553203176, 37.81569958979122, 23.665281983069725, -10.165609619407345,
-29.980059833502956, -10.101544762314273, 23.80872956937502, 38.068392349956305,
38.9815226439466, 40.84681874826839, 29.275804394700835, -2.1218503238029327,
-19.74311684413749, 2.003873391532892, 37.38594694370859, 52.66438629393816,
54.10483202536709, 55.9869996216546, 43.92362896481404, 11.54571373567099,
-7.5032088262144345, 12.426708932943658, 45.675307507552226, 58.588863001086324,
57.5264416569707, 56.865724020650106, 42.31787863965018, 7.608701920363714,
-13.531025437898329, 4.6246147539865285, 36.47721675320182, 48.41795021358077,
46.83207731063111, 46.103721796381876, 31.93051362557835, -1.9946648401653508,
-21.991905978314, -2.4019562470204496, 31.097150636103954, 44.80736430958389,
45.01890890257505, 46.0190717478295, 33.410946016665044, 0.7991827133635958,
-18.214489650450446, 1.9643874497834104, 35.60914747567561])

Plot the timeseries.

In [ ]:
time = np.arange(0,1, 1/400)

make_plot_1(time,data)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

Use the procedures outlined in Lesson 5 to examine the frequencies composing this timeseries.

First, figure out the step size (data were collected at 400 Hz).

In [ ]:
1/400

A data point is collected every 0.0025 seconds.

Start assessing. Start at 1 Hz

In [ ]:
amp_complex = np.exp(2*np.pi * 1j * time) 
amplitude_complex = data * amp_complex
spectrum = sum(amplitude_complex)*(1/400)/ 0.5
round(abs(spectrum),5)

0

No activity at 1 Hz.

Try 2 Hz.

In [ ]:
amp_complex = np.exp(2*np.pi * 2j * time) 
amplitude_complex = data * amp_complex
spectrum = sum(amplitude_complex)*(1/400)/ 0.5
round(abs(spectrum),5)

There is 2 Hz activity.

Try 3 Hz.

In [ ]:
amp_complex = np.exp(2*np.pi * 3j * time) 
amplitude_complex = data * amp_complex
spectrum = sum(amplitude_complex)*(1/400)/ 0.5
round(abs(spectrum),5)

0

No 3 Hz activity.

Try 4 Hz.

In [ ]:
amp_complex = np.exp(2*np.pi * 4j * time) 
amplitude_complex = data * amp_complex
spectrum = sum(amplitude_complex)*(1/400)/ 0.5
round(abs(spectrum),5)

There is 4 Hz activity.

HOLD ON!!! How many frequencies do we need to assess before we stop looking?

Take another look at the timeseries.

In [ ]:
time = np.arange(0,1, 1/400)

make_plot_1(time,data)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

There is high-frequency activity. Should we keep assessing until we find activity at higher frequencies?

Answer:
    
Using the above plug-and-chug method to determine the frequencies composing the timeseries will work, but it's often too much work. Instead, we want to automate the process, assessing activity in 1 Hz
steps until we get to the end. How do you know when you have reached the end?

Answer:
    
Nyquist's Rule tells us that, if the data are digitized at 400 Hz, then activity up to 200 Hz is accurately
represented in the timeseries. So, if we did this the long way, we would look for activity from 0 Hz (DC
value, i.e. average level across the entire epoch) to 200 Hz. Because we are examining a 1-second epoch, our frequency resolution is the reciprocal
of the epoch length. Thus, the frequency resolution is 1/1 sec = 1 Hz. As such, doing this the long way,
we'd need to perform 201 computations. (Do you know why it’s 201 and not 200? Remember that 0 Hz
is one of the “frequencies” - essentially representing the average value (different from zero) of the
overall timeseries. So, in the range 0 to 200, inclusive of the endpoints, there are 201 integers.)

Continue to the next section where we will examine a much better way to determine the frequencies
composing a timeseries.

## <font color=red>DSP.06.B2) The Fourier Transform (FT)</font>

### <font color=red>DSP.06.B2.a) The Fourier Transform</font>

We previously noted that a timeseries that indicates voltage or some other magnitude parameter as a function of
time is said to be a representation "in the time domain" (or distance in space; in fact, everything we say here about time in the context of the Fourier Transform applies to space as well, but we'll talk about time for simplicity). An alternative representation of the same
information is based on the principle that any waveform that is "stationary" (i.e., from which long-term
trends or changes in level have been removed and in which the frequency components do not change
in amplitude or phase over time) may be represented as the sum of a set of sinusoidal waveforms, each
of a different frequency and having an associated amplitude (or power) and phase. We emphasize that this applies to any waveform - not just waveforms actually composed of sinusoids.

(About that term "power"... In generic signal-processing contexts, magnitude, amplitude, voltage, and power are often used interchangeably. In the world of electrical phenomena, voltage is a unit of amplitude, and power is proportional to the square of voltage. Fourier analysis most often produces frequency magnitude values that technically speaking are power values. No need to differentiate among all of those terms here.)

This principle (the
Fourier Theorem) is the basis of Fourier analysis, which is a method for determining a set of sinusoids - their frequencies, amplitudes, and phases - that model of the timeseries. (This model accurately represents the timeseries, in the sense that subsequently applying an Inverse Fourier Transform (IFT) will recover it. That doesn't mean that the time series actually consisted of thos frequencies, amplitudes, and phases - only that it can be fully represented with such a model.) This representation of a signal is said to be "in the
frequency domain".

By now, you should be comfortable looking at frequency spectrum plots. Spectrum plots show the
frequency content of a timeseries and in the previous lessons were in fact obtained by applying a
Fourier Transform to the timeseries data. (Now you know our little secret.) The Fourier Transform is
used in acoustics when one wants to know (or at least model with sinusoids) the frequency content of a sound signal, in the neurosciences
when one wants to know (or at least model with sinusoids) the frequency content of brain activity, and in photography when
once wants to know (or at least model with sinusoids) the spatial frequency content of a photograph.

It has many other uses across many
fields. In fact, if you’re not careful, you could become very enamored of thinking about aspects of daily
life in terms of frequencies and how a Fourier transform might represent something, such as cloud
patterns, traffic backed up at a stoplight, the bumps on your tongue.... Fourier can handle them all!

### <font color=red>DSP.06.B2.b) The Fourier Transform Equation for Continuous Functions</font>

In Lesson 1, we saw that sine and cosine waves can be summed together to create almost any timeseries.
In Lesson 5, we saw that we can determine the amount of activity at a specific frequency within
a timeseries by multiplying the timeseries by sine and cosine waves of that frequency and then obtaining
the magnitude of the sine and cosine components. We additionally saw that complex exponentials
more elegantly determine the magnitude and phase of activity at a specific frequency in a given timeseries.
The Fourier Transform is just this use of complex exponentials to determine activity at specific
frequencies. However, rather than manually looking for activity one frequency at a time, the Fourier transform 'scans' the timeseries to examine activity at all possible frequencies.
(Well, in practice “all” is a bit of an exaggeration. There are limitations on what frequencies can be
found. We’ll discuss that after we present the ideal case.)

For a continuous signal or function f(t), the Fourier Transform is often defined as

$F( ω ) = \frac{\int_{-∞}^{∞}f(t)e^{i ω t}dt}{\sqrt{2\pi}}$

where ⅈ is the imaginary unity number (i.e., the square root of -1), and ω is the range of angular frequencies associated with the
signal (i.e., the frequency content of the signal). As ω changes, activity at each frequency is assessed.
F(ω) is the transfer function and tells us what to multiply each frequency ω by to model activity at that
frequency in the dataset. Because you successfully completed the earlier lessons, this formula should
not hold any surprises.

There are other common notations for the Fourier transform, and depending on the convention there may or may not be a term in front of the
integral to scale the data. In this introduction to the Fourier Transform, we will not consider these issues. However, if you are interested, take a
look online. We've found that different fields have different conventions about such things. 
        
When working with time and the signal f(t), one is working in the time domain, and the variables are real. When working with angular frequency and the Fourier transform F(ω) one is working in the frequency domain and F(ω) is complex (angular frequency is the rate at which the change in rotation takes place, or the rate at which change in the sinusoidal waves occurs).

### <font color=red>DSP.06.B2.c) The Discrete Fourier Transform (FT)</font>

The equation $F( ω ) = \frac{\int_{-∞}^{∞}f(t)e^{i ω t}dt}{\sqrt{2\pi}}$ involves an integral (a calculus thing) and is thus appropriate with continuous data.
However, more often than not we are working with a discrete signal, where the signal is digitized at a
particular sampling frequency. With discrete timeseries, the above equation is modified. Here is an
equation for the Discrete Fourier Transform (DFT):
    
$F[k Δf] =  \frac{\sum_{n=0}^{N-1}e^{i(2 \pi k Δf) (nΔt)}}{\sqrt{N}}$ for k = 0,1,2,..., N-1

The summation sign is used here because discrete rather than continuous data are examined. But the ideas are the same.
Also, as with the continuous form of the Fourier Transform, other definitions of the discrete Fourier transform are used in some scientific and
technical fields:
http://reference.wolfram.com/mathematica/tutorial/FourierTransforms.html
    
The relevant variables in the equation are:
    
N = total number of discrete samples

t = total sampling time [t = total number of points/sampling frequency fs; i.e., points / (points/second) = seconds]

Δf = frequency step size [Δf = 1/t; this is the set of frequencies that will be used to model the timeseries]

Δt = time increment between samples [Δt = t/N in seconds]

(We can also note that the sampling frequency is defined as fs = 1/Δt = N/t.)

Activity is assessed at every 1/t Hz, where 1/t is defined as the fundamental frequency. Thus, if the Fourier
Transform is applied to a 2-second epoch, the fundamental frequency is 1/2 = 0.5 Hz . As such, oscillatory
activity is examined in 0.5 Hz steps (i.e., 0, 0.5, 1.0, 1.5, ...., Nyquist frequency). If the Fourier Transform
is instead applied to a 10-second epoch, the fundamental frequency is 1/10 = 0.01 Hz . As such, oscillatory
activity is examined in 0.01 Hz steps (i.e., 0, 0.01, 0.02, ...., Nyquist frequency).

Note that the fundamental frequency is determined entirely by the length of the time series to be analyzed. It doesn't depend at all on the sampling frequency. The samping frequency determines how high the spectrum goes - it goes to the Nyquist frequency, which is half the sampling frequency. The length of the time series determines the width of each bin in the spectrum.

### <font color=red>DSP.06.B2.d) The Discrete Fourier Transform (FT) in action</font>

Look at how the Discrete Fourier Transform works.

In [ ]:
time = np.arange(0,1,1/40)
timeseries =  np.sin(2*np.pi * 4 * time) + 5 * np.sin(2*np.pi * 8 * time + np.pi/4)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

This is a 1-second epoch, with 40 total points. As such, the sample rate is 40 Hz (= fs), so the Nyquist
Frequency is 20 Hz.

Recalling our definitions (N = total number of discrete samples taken, t = total sampling time to be analyzed, Δt = time
increment between samples [Δt = t/N ], and Δf = frequency step size [Δf = 1/t ], solve for the following
values:
                           
t = 40/40 = 1 (i.e., the number of points sampled in one second is equal to the total points displayed).
                           
Δt = t/N = 1/40 = .025 sec = 25 ms (i.e., a sample is obtained every 25 ms)
                           
Δf = 1/t = 1 (1 (1× 1/1 )) = 1 Hz (i.e., activity is assessed in 1 Hz steps)
                           
(We can also note that fs = 1/Δt = N/t = 1/0.025 = 40 (1 (1× 1/1 )) = 40 Hz.)
                           
Use the Discrete Fourier Transform (DFT) to produce a frequency spectrum plot.
                           
First, take a look at the timeseries values.

In [ ]:
timeseries

Now, use the Python 'fft' function to calculate the Discrete Fourier Transform (DFT). (Pretty handy!)

In [ ]:
yf = fft(timeseries)
yf2 = []
for xx in yf:
   yf2.append(round_complex(xx))
yf2

Notice that the non-zero DFT values are complex numbers. The first value is the DC offset (i.e., the mean of the entire epoch being analyzed; here, zero, because there was no DC offset), followed by one value for each sampled frequency. This is just what we
did in Lesson 5, except that the Python fft function 'automatically' assesses sinusoidal
activity at all possible frequencies (well, not ALL - this is one of the limitations we hinted at above - we
will see below just what possible frequencies are assessed).

As we learned in Lesson 5, to compute the frequency spectrum, we need the magnitude of each number (the amount of activity at each frequency).
We know that the modulus of a complex number z = x + $i$y is given by |z| = $\sqrt{x^2 + y^2}$ .

Now plot the magnitude values.

In [ ]:
sample_rate = 40
duration = 1

n = sample_rate * duration
Nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(20,0,'Hz',fontsize=15)
plt.show()

That looks a little odd. Take another look at the complex valued output of the DFT.

In [ ]:
yf2

Every complex number above has both a positive and negative imaginary term. For example, after
skipping the first complex value (the DC value), the complex number for the 4 Hz activity in the fifth
position and fourth to last position are opposite in sign - see for yourself: -20j and 20j.

In [ ]:
abs(yf2[4])

In [ ]:
abs(yf2[-4])

The same value. This makes sense, because we're taking the absolute value of each complex term.

Because information in the first and last half of the DFT is identical, people usually plot only one half of
the DFT values.

In [ ]:
make_plot_freq_1(timeseries,40)

# Adding text to the figure
plt.text(20,0,'Hz',fontsize=15)
plt.show()

At this point we just need to come up with the correct x-axis values. Look the original output one more
time

In [ ]:
yf2[0:Nhalf]

The first number represents the DC offset (the mean). The magnitude of each of the subsequent complex
numbers represents the amount of power at a particular frequency. The frequency each value represents
can be found by calculating the frequency increment Δf, calculated as 1/T (Δf is also called the
frequency resolution or the fundamental frequency). In the current example, Δf = 1/1 second = 1 Hz. Thus, each complex value defines the
relative contribution of each frequency to the signal, in 1 Hz steps (1 Hz, 2 Hz, 3 Hz, 4 Hz, ...).

Recalling the DFT formula     
$F[k Δf] =  \frac{\sum_{n=0}^{N-1}e^{i(2 \pi k Δf) (nΔt)}}{\sqrt{N}}$ for k = 0,1,2,..., N-1, 
we can now see:


for k = 1, F(kΔf) is the Discrete Fourier transform at the first harmonic frequency Δf

for k = 2, F(kΔf) is the Discrete Fourier transform at the second harmonic frequency 2Δf

...until
for k = 20, F(kΔf) is the Discrete Fourier transform at the 20th harmonic frequency 20Δf

Now that we know Δf, we can plot the power spectrum with the correct x axis.

In [ ]:
make_plot_freq_1(timeseries,40)

# Adding text to the figure
plt.text(20,0,'Hz',fontsize=15)
plt.show()

The magnitude (on the y axis) at each point (on the x axis) indicates the importance of that frequency in
the signal. We see a peak at 4 Hz with a magnitude value of 20 and an 8 Hz signal with an amplitude of 100.
Look again at the original timeseries. (Reminder: this doesn't confirm that the original data consisted of 4 Hz and 8 Hz components. It just means that the original data can be successfully modeled as the sum of 4 Hz and 8 Hz sinusoids.)

In [ ]:
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

But in this case, our DFT does indicate what was truly in the original timeseries: activity at 4 and 8 Hz.

Some confusion can come from the terminology of the Fourier Transform. In reality, the data are not
"transformed". The original data remain, but new vectors are created that describe a set of sine waves. There is a power or amplitude vector, containing one such magnitude value for each harmonic,
and a phase vector, again with one value for each harmonic. (So far, we've plotted just the magnitude vector, typically called the power or amplitude spectrum. We haven't plotted the phase spectrum, but one can do that. It shows the phase of each of the frequencies in the power spectrum.) Thus, the DFT as a computational procedure
figures out a set of sine waves with frequency, magnitude, and phase characteristics that, when summed, describe a vector of timeseries data.

### <font color=red>DSP.06.B2.e) Another example of the DFT</font>

Run the code.

In [ ]:
time = np.arange(0,1,1/50)
timeseries = 0.5 * np.sin(2*np.pi * 2 * time) + 3 * np.sin(2*np.pi * 5 * time)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

Check above to see that we are sampling at 50 Hz (= fs). The total number of points is 50 (= points) = N.

Recalling our definitions (N = total number of discrete samples taken, t = total sampling time, Δt = time
increment between samples [Δt = t/N], and Δf = frequency step size [Δf = 1/t], solve for the following
values:
                           
t = 50/50 = 1 second (because the number of points sampled in one second is equal to the total number of
points displayed)
                           
Δt = t/N = 1/50 = 0.02 sec = 20 ms
                           
Δf = 1/t = 1 (1 (1× 1/1 )) = 1 Hz (i.e., activity is assessed in 1 Hz steps)
                           
(We can also note that fs = 1/Δt = N/t = 1/0.02 = 50 (1 (1× 1/1 )) = 50 Hz.)
                           
For the DFT, grab the y values.

In [ ]:
timeseries

Use the Python 'fft' function to calculate the DFT. 

In [ ]:
from scipy.fft import fft, fftfreq
yf = fft(timeseries)
yf2 = []
for xx in yf:
   yf2.append(round_complex(xx))
yf2

The DFT numbers are complex, and there is one value for each frequency. (Remember - that includes 0 Hz, which is the mean of the entire timeseries vector, often called the DC offset.)

We want the magnitude for each of the above complex numbers, so calculate the modulus as described
above.

In [ ]:
np.abs(yf2)

Plot the above data.

In [ ]:
sample_rate = 50
duration = 1

n = sample_rate * duration
nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(26,0,'Hz',fontsize=15)
plt.show()

Remember, the first and second halves of the DFT output contain the same information. Throw out the
second half of the data and plot just the first half.

In a more advanced digital signal processing class, you'd learn why the Fourier output has this structure. For now, however, all we need to
do is note that this is how the Fourier output is structured.

In [ ]:
make_plot_freq_1(timeseries,50)

# Adding text to the figure
plt.text(26,0,'Hz',fontsize=15)
plt.show()

Look the original output one more time

In [ ]:
yf2[0:nhalf]

The first complex value is the DC offset = the mean. The magnitude of each of the subsequent complex numbers
represents the amount of power at a particular frequency. The frequency represented by each
value is found by calculating the frequency increment Δf, calculated as 1/t ( Δf is also called the frequency
resolution). In the current example Δf = 1/1 second = 1 Hz. Thus, each of the complex numbers
defines the relative contribution to the signal by each discrete frequency in 1 Hz steps (i.e., 1 Hz, 2 Hz, 3
Hz, 4 Hz, ...).

Recalling the DFT formula     
$F[k Δf] =  \frac{\sum_{n=0}^{N-1}e^{i(2 \pi k Δf) (nΔt)}}{\sqrt{N}}$ for k = 0,1,2,..., N-1, 
we can now see that

for k = 1, F(kΔf) is the Discrete Fourier Transform at the first harmonic frequency Δf

for k = 2. F(kΔf) is the Discrete Fourier Transform at the second harmonic frequency 2Δf

...until
for k = 25, F(kΔf) is the Discrete Fourier Transform at the 25th harmonic frequency 25Δf

And now that we know Δf, we can plot the power spectrum with the correct x-axis values.

In [ ]:
make_plot_freq_1(timeseries,50)

# Adding text to the figure
plt.text(26,0,'Hz',fontsize=15)
plt.show()

A plot of the magnitude of the DFT output as a function of frequency is called the frequency spectrum.
The amplitude at each point indicates the importance of that frequency in the signal.

Note that the Fourier Transform method doesn’t care what frequencies are in the signal - how many
different frequencies, their absolute or relative amplitude, whether the signal is voltages or temperatures
or densities or number of scoops of ice cream... So you can start to see the generality of this
method.

In this example, there are two peaks - one at 2 Hz and one at 5 Hz. In addition, the 5 Hz activity is
larger than the 2 Hz activity. Why?

Answer: Look at the original output one more time.

In [ ]:
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

If you study the plot, looking for 2 Hz variation and 5 Hz variation, you'll notice that the amplitude of the 2 Hz sine wave is 12.5 and the amplitude of the 5 Hz sine wave is 75. This
explains why the 5 Hz peak is larger in the frequency spectrum.

## <font color=red>DSP.06.B3) Going the Opposite Way - The Inverse Fourier Transform</font>

### <font color=red>DSP.06.B3.a) Examining the Discrete Fourier Transform results</font>

In Lesson 1, we saw how to combine sine waves to get any timeseries (e.g., square waves, saw-tooth
waves). In the preceding sections of the present lesson, we've seen that the DFT is an elegant way to
determine the specific frequencies composing a stationary timeseries. Here’s an important complementary fact: Theoretically, if someone sent us DFT data, we should be able to use this information to
recreate the original time series. We just need to sum the relevant frequencies, adjusting each frequency
by the DFT magnitude and phase values.

Give it a try. Apply the Discrete Fourier Transform to the timeseries below.

In [ ]:
time = np.arange(0,1,1/250)
timeseries = 5 * np.sin(2*np.pi * 10 * time) 

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
sample_rate = 250
duration = 1

n = sample_rate * duration
nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

yf = yf[0:nhalf]
xf = xf[0:nhalf]

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(126,0,'Hz')
plt.show()

Take a look at the DFT frequency and then magnitude values.

In [ ]:
xf

In [ ]:
np.round(np.abs(yf),4)

For each pair of values, the first value provides the frequency information, and the second value the
magnitude information. We see that there is activity only at 10 Hz.

### <font color=red>DSP.06.B3.b) Examining the DFT results</font>

Take a look at this timeseries.

In [ ]:
time = np.arange(0,1,1/250)
timeseries = np.sin(2*np.pi * 5 * time) +  6 * np.sin(2*np.pi * 10 * time) + 4 * np.sin(2*np.pi * 15 * time)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
sample_rate = 250
duration = 1

n = sample_rate * duration
nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

yf = yf[0:nhalf]
xf = xf[0:nhalf]

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(126,0,'Hz')
plt.show()

Look at the DFT frequency and magnitude values.

In [ ]:
xf

In [ ]:
np.round(np.abs(yf),4)

For each pair of values, the first value provides the frequency information and the second value the
magnitude information. We see that there is activity at 5, 10, and 15 Hz. This matches what we know
about the function used to create this time series: f[x_] = Sin [5x×2$π$] + Sin[10x×2$π$] + Sin[15x×2$π$].

### <font color=red>DSP.06.B3.c) Creating a timeseries from the DFT results</font>

Here is another timeseries.

In [ ]:
time = np.arange(0,1, 1/250)
timeseries = np.sin(2*np.pi * 5 * time) +  np.sin(2*np.pi * 10 * time) + np.sin(2*np.pi * 15 * time) +\
              np.sin(2*np.pi * 27 * time) + np.sin(2*np.pi * 58 * time)

make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
sample_rate = 250
duration = 1

n = sample_rate * duration
nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

yf = yf[0:nhalf]
xf = xf[0:nhalf]

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(126,0,'Hz')
plt.show()

Take a look at the DFT frequency and magnitude values.

In [ ]:
xf

In [ ]:
np.round(np.abs(yf),4)

This is getting complicated, but take a close look. There is activity at 5, 10, 15, 27, and 58 Hz. Let's use
this information to recreate the timeseries (from scratch, not from the Inverse Fourier Transform) IFT.

First, create the individual sine waves.

In [ ]:
fiveHz = np.sin(2*np.pi * 5 * time)
tenHz = np.sin(2*np.pi * 10 * time)
fifteenHz = np.sin(2*np.pi * 15 * time)
twentysevenHz = np.sin(2*np.pi * 27 * time)
fiftyeightHz = np.sin(2*np.pi * 58 * time)

Now plot each timeseries.

In [ ]:
time = np.arange(0,1, 1/250)

# Plotting time vs amplitude using plot function from pyplot
plt.plot(time, fiveHz)
plt.plot(time, tenHz)
plt.plot(time, fifteenHz)
plt.plot(time, twentysevenHz)
plt.plot(time, fiftyeightHz)

plt.margins(x=0, y=0)
plt.axhline(y=0, color='k')
plt.tick_params(labelbottom = False, bottom = False)

# Adding text to the figure
plt.text(1,0,'1 sec')

# Finally displaying the plot
plt.show()

The above plot shows an overlay of the five timeseries. Not very informative.

Instead, add the functions together and then plot the summed result along with the original timeseries.

In [ ]:
time = np.arange(0,1, 1/250)
combined = fiveHz + tenHz + fifteenHz + twentysevenHz + fiftyeightHz

make_plot_1(time,combined)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

As expected, they are the same. This shows that it's possible to move from the time domain to the
frequency domain, and then back to the time domain, recreating the timeseries from the spectrum.

### <font color=red>DSP.06.B3.d) Creating a timeseries from the DFT results</font>

Now something a little more complicated. Take a look at a plot of the timeseries (1 second, digitized at
250 Hz) and then also the DFT magnitude values for each frequency for a new timeseries.

In [ ]:
tenHz = np.array([0.7071067811865475, 0.8607420270039436, 0.9602936856769431,
0.9995065603657316, 0.9759167619387474, 0.8910065241883679, 0.7501110696304596,
0.5620833778521306, 0.33873792024529137, 0.09410831331851431, -0.15643446504023087,
-0.3971478906347806, -0.6129070536529764, -0.7901550123756904, -0.9177546256839811,
-0.9876883405951378, -0.99556196460308, -0.9408807689542255, -0.8270805742745618,
-0.6613118653236518, -0.45399049973954675, -0.21814324139654256,
0.03141075907812829, 0.2789911060392293, 0.5090414157503713, 0.7071067811865475,
0.8607420270039436, 0.9602936856769431, 0.9995065603657316, 0.9759167619387474,
0.8910065241883679, 0.7501110696304596, 0.5620833778521306, 0.33873792024529137,
0.09410831331851431, -0.15643446504023087, -0.3971478906347806, -0.6129070536529764,
-0.7901550123756904, -0.9177546256839811, -0.9876883405951378, -0.99556196460308,
-0.9408807689542255, -0.8270805742745618, -0.6613118653236518, -0.45399049973954675,
-0.21814324139654256, 0.03141075907812829, 0.2789911060392293, 0.5090414157503713,
0.7071067811865475, 0.8607420270039436, 0.9602936856769431, 0.9995065603657316,
0.9759167619387474, 0.8910065241883679, 0.7501110696304596, 0.5620833778521306,
0.33873792024529137, 0.09410831331851431, -0.15643446504023087, -0.3971478906347806,
-0.6129070536529764, -0.7901550123756904, -0.9177546256839811, -0.9876883405951378,
-0.99556196460308, -0.9408807689542255, -0.8270805742745618, -0.6613118653236518,
-0.45399049973954675, -0.21814324139654256, 0.03141075907812829, 0.2789911060392293,
0.5090414157503713, 0.7071067811865475, 0.8607420270039436, 0.9602936856769431,
0.9995065603657316, 0.9759167619387474, 0.8910065241883679, 0.7501110696304596,
0.5620833778521306, 0.33873792024529137, 0.09410831331851431, -0.15643446504023087,
-0.3971478906347806, -0.6129070536529764, -0.7901550123756904, -0.9177546256839811,
-0.9876883405951378, -0.99556196460308, -0.9408807689542255, -0.8270805742745618,
-0.6613118653236518, -0.45399049973954675, -0.21814324139654256,
0.03141075907812829, 0.2789911060392293, 0.5090414157503713, 0.7071067811865475,
0.8607420270039436, 0.9602936856769431, 0.9995065603657316, 0.9759167619387474,
0.8910065241883679, 0.7501110696304596, 0.5620833778521306, 0.33873792024529137,
0.09410831331851431, -0.15643446504023087, -0.3971478906347806, -0.6129070536529764,
-0.7901550123756904, -0.9177546256839811, -0.9876883405951378, -0.99556196460308,
-0.9408807689542255, -0.8270805742745618, -0.6613118653236518, -0.45399049973954675,
-0.21814324139654256, 0.03141075907812829, 0.2789911060392293, 0.5090414157503713,
0.7071067811865475, 0.8607420270039436, 0.9602936856769431, 0.9995065603657316,
0.9759167619387474, 0.8910065241883679, 0.7501110696304596, 0.5620833778521306,
0.33873792024529137, 0.09410831331851431, -0.15643446504023087, -0.3971478906347806,
-0.6129070536529764, -0.7901550123756904, -0.9177546256839811, -0.9876883405951378,
-0.99556196460308, -0.9408807689542255, -0.8270805742745618, -0.6613118653236518,
-0.45399049973954675, -0.21814324139654256, 0.03141075907812829, 0.2789911060392293,
0.5090414157503713, 0.7071067811865475, 0.8607420270039436, 0.9602936856769431,
0.9995065603657316, 0.9759167619387474, 0.8910065241883679, 0.7501110696304596,
0.5620833778521306, 0.33873792024529137, 0.09410831331851431, -0.15643446504023087,
-0.3971478906347806, -0.6129070536529764, -0.7901550123756904, -0.9177546256839811,
-0.9876883405951378, -0.99556196460308, -0.9408807689542255, -0.8270805742745618,
-0.6613118653236518, -0.45399049973954675, -0.21814324139654256,
0.03141075907812829, 0.2789911060392293, 0.5090414157503713, 0.7071067811865475,
0.8607420270039436, 0.9602936856769431, 0.9995065603657316, 0.9759167619387474,
0.8910065241883679, 0.7501110696304596, 0.5620833778521306, 0.33873792024529137,
0.09410831331851431, -0.15643446504023087, -0.3971478906347806, -0.6129070536529764,
-0.7901550123756904, -0.9177546256839811, -0.9876883405951378, -0.99556196460308,
-0.9408807689542255, -0.8270805742745618, -0.6613118653236518, -0.45399049973954675,
-0.21814324139654256, 0.03141075907812829, 0.2789911060392293, 0.5090414157503713,
0.7071067811865475, 0.8607420270039436, 0.9602936856769431, 0.9995065603657316,
0.9759167619387474, 0.8910065241883679, 0.7501110696304596, 0.5620833778521306,
0.33873792024529137, 0.09410831331851431, -0.15643446504023087, -0.3971478906347806,
-0.6129070536529764, -0.7901550123756904, -0.9177546256839811, -0.9876883405951378,
-0.99556196460308, -0.9408807689542255, -0.8270805742745618, -0.6613118653236518,
-0.45399049973954675, -0.21814324139654256, 0.03141075907812829, 0.2789911060392293,
0.5090414157503713, 0.7071067811865475, 0.8607420270039436, 0.9602936856769431,
0.9995065603657316, 0.9759167619387474, 0.8910065241883679, 0.7501110696304596,
0.5620833778521306, 0.33873792024529137, 0.09410831331851431, -0.15643446504023087,
-0.3971478906347806, -0.6129070536529764, -0.7901550123756904, -0.9177546256839811,
-0.9876883405951378, -0.99556196460308, -0.9408807689542255, -0.8270805742745618,
-0.6613118653236518, -0.45399049973954675, -0.21814324139654256,
0.03141075907812829, 0.2789911060392293, 0.5090414157503713])

tenHz

In [ ]:
time = np.arange(0,1, 1/250)
make_plot_1(time,tenHz)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
make_plot_freq_1(tenHz,250)

# Adding text to the figure
plt.text(126,0,'Hz',fontsize=15)
plt.show()

The frequency spectrum plot shows a clear peak at 10 Hz. An examination of the DFT frequency and
magnitude values confirms 10 Hz activity.

Make a 10 Hz timeseries and then overlay the original and new timeseries.

In [ ]:
time = np.arange(0,1, 1/250)
data = np.sin(2*np.pi * 10 * time)
make_plot_1(time,data)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
make_plot_2(time,tenHz,'b',time,data,'r')
plt.text(1,0,'1 sec',fontsize=15)
plt.show()  

Not exactly the same. The two timeseries differ in phase. What went wrong?

Answer:
    
To recreate the original timeseries, we need to know more than the frequency and magnitude values.
Remember, a complete description of a sine wave includes information about the frequency, the
magnitude, and the phase. So far, we've used only frequency and magnitude information. Let's look at
the phase information and see if that helps.

We can obtain phase information from complex numbers using the Python 'arctan' function.
Let's take a look at the DFT output on the original data.

In [ ]:
yf = fft(tenHz)
yf

We need to obtain the phase information from the complex number representing 10 Hz activity.

Grab that point (the 10th point from the end) and obtain the phase information.

In [ ]:
point = yf[-10]
point_x = np.real(point)
point_y = np.imag(point)
phaseradians = round(np.arctan(point_x/point_y),5)
phasedegrees = round(phaseradians * 180 / np.pi,2)
phasedegrees

Use this information to create a 10 Hz sin wave with the above phase delay.

In [ ]:
time = np.arange(0,1, 1/250)
data = np.sin(2*np.pi * 10 * time + np.pi/4)
make_plot_1(time,data)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
make_plot_2(time,tenHz,'b',time,data,'r')
plt.text(1,0,'1 sec',fontsize=15)
plt.show() 

Looks good now.

## <font color=red>DSP.06.B4) The Inverse Fourier Transform</font>

### <font color=red>DSP.06.B4.a) Inverse Fourier Transform</font>

You now know how to examine the DFT output to obtain the information needed to recreate the original
timeseries, via the IFT. Although you can easily recreate the original timeseries directly when there are only a few
frequencies, doing so becomes difficult when there is activity at many frequencies.

Take a look at the DFT of the timeseries below.

In [ ]:
timeseries = 0.6 * np.sin(2*np.pi * 5 * time) +  5.6*np.sin(2*np.pi * 10 * time+np.pi/3) + np.sin(2*np.pi * 15 * time+np.pi/6) +\
              0.9*np.sin(2*np.pi * 27 * time) + np.sin(2*np.pi * 58 * time+np.pi/5)+\
            0.3 * np.sin(2*np.pi * 4 * time) +  7.6*np.sin(2*np.pi * 53 * time+np.pi/3) + np.sin(2*np.pi * 22 * time+np.pi/6) +\
              0.9*np.sin(2*np.pi * 43 * time) + np.sin(2*np.pi * 33 * time+np.pi/5)
            
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
sample_rate = 250
duration = 1

n = sample_rate * duration
nhalf = round(n/2)

yf = fft(timeseries)
xf = fftfreq(n, 1 / sample_rate)

yf = yf[0:nhalf]
xf = xf[0:nhalf]

plt.plot(xf, np.abs(yf))

# Adding text to the figure
plt.text(126,0,'Hz',fontsize=15)
plt.show()

Lots going on there. As demonstrated above, it would be possible to look at the DFT output (the magnitude and phase for each frequency) to reconstruct the original
timeseries. However, there is an easier way. The DFT takes you from the time to the frequency domain.
To move from the frequency to the time domain, we compute the Inverse Fourier Transform.

Here is the Fourier Transform

$F( ω ) = \frac{\int_{-∞}^{∞}f(t)e^{i ω t}dt}{\sqrt{2\pi}}$

and here is the Inverse Fourier Transform

$F( t ) = \frac{\int_{-∞}^{∞}F(ω)e^{-i ω t}dω}{\sqrt{2\pi}}$

where in both cases ⅈ is the imaginary unity number, and ω is the range of frequencies associated with
the signal (i.e., the frequency content of the signal). When we calculate the forward Fourier transform, we integrate across time. The inverse integrates across the component frequencies. You can see that the two formulas are very similar. You probably noticed that the positions of ω and t are reversed.  Also, the sign of the exponent changes. Otherwise, the two formulas are basically...the same formula. This highlights the bidirectionaliy of the Fourier method. Underlying that is the intertranslatability of time-domain and frequency-domain representations of a phenomenon.

We can note the following:
    
A Discrete Fourier Transform converts a digitally represented signal from the time domain to the frequency
domain.

An Inverse Fourier Transform does the converse.

No information is lost in either transform - each is simply a way to represent the original vector of data.

You can easily calculate the Inverse Fourier Transform with Python.

Here are the original data and the Fourier Transform output.

In [ ]:
timeseries

In [ ]:
yf2 = []
for xx in yf:
   yf2.append(round_complex(xx))
yf2

In [ ]:
from scipy.fft import fft, ifft, fftfreq

yf = fft(timeseries)
timeseries2 = ifft(yf)
timeseries2

Looks the same as the original timeseries. To confirm, plot the data and then overlay the original and
the recreated timeseries

In [ ]:
make_plot_1(time,timeseries2)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

In [ ]:
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)

plt.show()

Exactly the same.

## <font color=red>DSP.06.B5) The Forward and Inverse Fourier Transform - Some Thoughts</font>

### <font color=red>DSP.06.B5.a) Time-domain and frequency-domain representations</font>

The interchangeability (more precisely, the intertranslatability) of time-domain and frequency-domain representations of a given waveform
bears emphasis. Consider a set of j sine waves. Given the stationarity assumption, the frequency,
amplitude, and the phase of each sine wave are constant throughout the analyzed epoch. At any particular
time during the epoch, the different sine waves may be at different points in their cycles. Summing
across the set of sine waves produces a single composite waveform in which the constituents may be
difficult to identify. One could digitize the composite waveform, describing it as a single vector of
values arranged in time. Alternatively, one could describe it with power and phase vectors (known
as the power and phase spectra) arranged in order by frequency. Either description--in the time
domain or in the frequency domain--completely specifies all of the information contained in the digitized
composite waveform. One description may be more tractable for a given set of analyses or more
intuitively appealing for a given question, but the same information is available in the two representations.

It should be noted that the fidelity of a Fourier analysis is limited by the extent to which modeling the
data as a sum of invariant sinusoids is appropriate to the raw data. Because Fourier-transformed data
in the frequency domain contain exactly the information in the original time-domain waveform, the
Fourier transform, in either direction, does not introduce distortion into the data. However, characterization
of the data via a set of sine waves does not ensure that the original phenomena were indeed
sinusoidal, that the original phenomenon was stationary over the epoch analyzed, or that scoring in terms of sine waves provides a high-fidelity representation of those
phenomena. The Fourier Theorem just says that we can create a sinusoidal model of any vector of data.

Quick terminology note: You may run across the term "Fast Fourier Transform" (FFT). It happens that, if one makes certain assumptions or adopts certain restrictions about one's data, one can derive some computational shortcuts to accomplish the DFT faster. Generally, FFT refers to suc a method for doing a DFT. A common restriction is that the length of the timeseries is a power of 2. That's one reason one sometimes sees a sampling rate of 512 or 1024; or, for say 1000 Hz sampling rate, an analysis epoch of 1.024 seconds (containing 1024 samples). Everything we've discussed here applies to those cases, but we've considered the general case - not limited e.g. to power-of-2 analysis epochs.